In [1]:
import pandas as pd
import warnings
import os
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [2]:
parent_dir = os.path.dirname(os.getcwd())
data_path = os.path.join(parent_dir, 'data', 'dataset')

In [3]:
dataset = pd.read_csv(os.path.join(data_path, 'dataset.csv'))
print(dataset.shape)

(1312248, 15)


**CHAMPION ROLE WINRATE**

In [4]:
champ_role_dataset = dataset.copy()
champ_role_dataset = pd.get_dummies(champ_role_dataset, columns=['WIN'])

In [5]:
champ_role_extracted = champ_role_dataset[['CHAMPION','ROLE', 'WIN_True','WIN_False']].reset_index(drop = True)

In [6]:
champ_role_extracted.head()

,CHAMPION,ROLE,WIN_True,WIN_False
0,Kassadin,top,0,1
1,Fiora,top,1,0
2,Yone,middle,0,1
3,Corki,middle,1,0
4,Kaisa,bottom,0,1


In [7]:
champ_role_extracted['champ_role'] = champ_role_extracted['CHAMPION'].str.cat(champ_role_extracted[['ROLE']], sep='_')
champ_role_extracted.champ_role.nunique()

772

In [8]:
# champ in role count distribution.
# Only keep those combinations which have atleast 15 records
counts = champ_role_extracted.champ_role.value_counts().reset_index(name='count')
counts = counts.query('count > 15')
counts

,index,count
0,Ezreal_bottom,35172
1,Kaisa_bottom,28125
2,Thresh_utility,26293
3,LeeSin_jungle,23512
4,Lulu_utility,20272
...,...,...
560,Leblanc_bottom,16
561,TwistedFate_jungle,16
562,Camille_bottom,16
563,Volibear_utility,16


In [9]:
champ_role_extracted = champ_role_extracted[champ_role_extracted.champ_role.isin(counts['index'])]
champ_role_extracted

,CHAMPION,ROLE,WIN_True,WIN_False,champ_role
0,Kassadin,top,0,1,Kassadin_top
1,Fiora,top,1,0,Fiora_top
2,Yone,middle,0,1,Yone_middle
3,Corki,middle,1,0,Corki_middle
4,Kaisa,bottom,0,1,Kaisa_bottom
...,...,...,...,...,...
1312243,Alistar,utility,1,0,Alistar_utility
1312244,LeeSin,jungle,0,1,LeeSin_jungle
1312245,FiddleSticks,jungle,1,0,FiddleSticks_jungle
1312246,Shen,top,0,1,Shen_top


In [10]:
champ_role_extracted = champ_role_extracted.groupby(['CHAMPION', 'ROLE']).apply(lambda x: (x['WIN_True'].sum()/(x['WIN_True'].sum() + x['WIN_False'].sum() )*100)).reset_index(name='WINRATE')

In [11]:
champ_role_extracted.describe()

,WINRATE
count,565.000000
mean,47.824775
std,6.930245
min,15.625000
25%,45.454545
50%,49.375975
75%,51.309397
max,72.000000


In [12]:
champ_role_extracted.to_csv(os.path.join(data_path, 'champ_role_wr.csv'), index = False)